In [ ]:
!nvidia-smi # reset until p100

Thu Apr 28 05:34:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
my_cluster = 0

clusters = [1, 1, 2, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 0, 2, 2]
species = []

my_species = 


In [ ]:
%%capture
! pip install deeplabcut==2.2.0.6
import deeplabcut

In [ ]:
! cp -r /content/drive/MyDrive/5561/baseline ./baseline

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/train_cropped.zip train_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/val_cropped.zip val_cropped.zip
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Zips/test_cropped.zip test_cropped.zip

In [ ]:
%%capture
! unzip train_cropped.zip
! unzip val_cropped.zip
! unzip test_cropped.zip

In [ ]:
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/train_annotation.json train_annotation.json
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/val_annotation.json val_annotation.json
! cp /content/drive/MyDrive/5561/OpenMonkeyChallenge/Annotations/test_prediction.json test_prediction.json

In [ ]:
import json

with open("train_annotation.json") as f:
  train_js = json.load(f)

with open("val_annotation.json") as f:
  val_js = json.load(f)

with open("test_prediction.json") as f:
  test_js = json.load(f)

In [ ]:
import numpy as np
species = list(np.unique([example['species'] for example in train_js['data']]))

In [ ]:
def get_cropped_name(name):
  return name[:-11] + "cropped_" + name[-11:]

get_cropped_name('train_0000000.jpg')

'train_cropped_0000000.jpg'

In [ ]:
def get_img_names(species):
  names = []
  for train in train_js['data']:
    if train['species'] == species:
      names.append(get_cropped_name(train['file']))
  for val in val_js['data']:
    if val['species'] == species:
      names.append(get_cropped_name(val['file']))
  return names

len(get_img_names("Gorilla"))

1778

In [ ]:
def get_img_names_test(species):
  names = []
  for test in test_js['data']:
    if test['species'] == species:
      names.append(get_cropped_name(test['file']))
  return names

get_img_names_test("Gorilla")[:10]

['test_cropped_0000112.jpg',
 'test_cropped_0000142.jpg',
 'test_cropped_0000245.jpg',
 'test_cropped_0000277.jpg',
 'test_cropped_0000298.jpg',
 'test_cropped_0000328.jpg',
 'test_cropped_0000338.jpg',
 'test_cropped_0000356.jpg',
 'test_cropped_0000396.jpg',
 'test_cropped_0000422.jpg']

In [ ]:
species_to_imgs = dict([(s, get_img_names(s)) for s in species])
species_to_imgs['Gorilla'][:10]

['train_cropped_0000036.jpg',
 'train_cropped_0000048.jpg',
 'train_cropped_0000101.jpg',
 'train_cropped_0000133.jpg',
 'train_cropped_0000146.jpg',
 'train_cropped_0000197.jpg',
 'train_cropped_0000217.jpg',
 'train_cropped_0000256.jpg',
 'train_cropped_0000294.jpg',
 'train_cropped_0000341.jpg']

In [ ]:
import os
import shutil

for s in species:
  shutil.copytree("baseline", f"baseline_{s}")

In [ ]:
for s in species:
  os.mkdir(f"test_cropped_{s}")

In [ ]:
def crop_df(df, imgs):
  return df[df.index.get_level_values(2).isin(imgs)]

In [ ]:
import pandas as pd

for s in species:
  dir = f"baseline_{s}/"
  csv = dir + "labeled-data/baseline/CollectedData_daniel.csv"
  h5 = dir + "labeled-data/baseline/CollectedData_daniel.h5"
  df = pd.read_hdf(h5)
  imgs = species_to_imgs[s]
  df_cropped = crop_df(df, imgs)
  df_cropped.to_csv(csv)
  df_cropped.to_hdf(h5, mode="w", key="label")

In [ ]:
# Move images

import os
import shutil

for s in species:
  imgs = species_to_imgs[s]
  for img in imgs:
    prefix = "train_cropped/" if "train" in img else "val_cropped/"
    shutil.move(prefix + img, f"baseline_{s}/labeled-data/baseline/{img}")

In [ ]:
for s in species:
  imgs = get_img_names_test(s)
  for img in imgs:
    shutil.move(f"test_cropped/{img}", f"test_cropped_{s}/{img}")

In [ ]:
def edit_snapshot_config(path_config_file, snapshot_prefix):
  shuffle = 1
  trainingsetindex = 0
  trainposeconfigfile, testposeconfigfile, snapshotfolder = deeplabcut.return_train_network_path(path_config_file, shuffle=shuffle, trainingsetindex=trainingsetindex)
  cfg_dlc = deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

  # Check if there are snapshots here
  snapshots_here = os.listdir(snapshot_prefix)

  if snapshots_here != []:
    snapshot_list = [snap.split(".")[0] for snap in snapshots_here]
    snapshot_list.sort(key = lambda x : int(x.split("-")[1]))
    snapshot = snapshot_list[-1]
    cfg_dlc["init_weights"] = snapshot_prefix + "/" + snapshot
    deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

In [ ]:
# Train and inference!

for s in species:
  already_trained = [f.split(".")[0][17:] for f in os.listdir("drive/MyDrive/5561/26-predictions")]
  if s in already_trained:
    continue

  path_config_file = f"/content/baseline_{s}/config.yaml"

  print(f"Training model for {s}")

  deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")

  snapshot_prefix = f"/content/drive/MyDrive/5561/26-snapshots/{s}"

  edit_snapshot_config(path_config_file, snapshot_prefix)
    
  deeplabcut.train_network(path_config_file, max_snapshots_to_keep=5, displayiters=100, saveiters=10000, maxiters=350000)
  deeplabcut.analyze_videos(path_config_file,
                            [f'/content/test_cropped_{s}'], 
                            videotype='.jpg', 
                            destfolder=f"/content/test_predictions_{s}")

  # save results
  shutil.make_archive(f"/content/drive/MyDrive/5561/26-predictions/test_predictions_{s}", 'zip', f"/content/test_predictions_{s}")